# <center>Big Data &ndash; Exercise 1 &ndash; Solutions</center>
## <center>Fall 2025 &ndash; Week 1 &ndash; ETH Zurich</center>

### Aims
- **After this exercise:** 
    - Understand the SQL language and its common query patterns.
    - Understand the 'table' data shape, normalization, and when they can (and should) be used.
    - be able to query data in tables with the SQL language.
- **Later in the semester:** 
    - Relate these language features and query patterns relative to other data shapes, technologies, and the languages designed to query them.
    - Understand when tables are not the appropriate shape for your data and when you can (and should) throw normalization away!

### Prerequisites
In this exercise, you will brush-up the fundamental concepts of relational databases and SQL. If you haven't taken an introductory databases course (or want to refresh your knowledge) we recommend you to read the following:

Garcia-Molina, Ullman, Widom: Database Systems: The Complete Book. Pearson, 2. Edition, 2008. (Chapters 1, 2, 3, and 6) [Available in the ETH Library] [[Online]](https://ebookcentral.proquest.com/lib/ethz/detail.action?pq-origsite=primo&docID=5832965) [[Selected solutions]](http://infolab.stanford.edu/~ullman/dscbsols/sols.html).

Or have a look at the recordings from Information Systems for Engineers - ETH Zurich, available on [[YouTube]](https://www.youtube.com/c/GhislainFournysLectures).

### Database Set-up
We will be once again working in the ExamMagicBox (you can find it in the following [[link]](https://polybox.ethz.ch/index.php/s/wa57XqDKkxRMb0q) if you have not downloaded it yet): please drag this Notebook in the folder. Just like last week, activate the docker container for the exercise sheet with `docker compose up`; please wait for the message `PostgreSQL init process complete; ready for start up` in the docker logs before proceeding! Alternatively you can start the Docker with `docker compose up -d` and wait for the command to execute: please note that you are creating the containers in the background this way. You can then type `docker compose down` when you are done.

As before, we set up our connection to the database and enable use of `%sql` and `%%sql`.

In [ ]:
%reload_ext sql
%sql postgresql://postgres:example@db

In [3]:
%%sql
SELECT version();

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


version
"PostgreSQL 16.2 (Debian 16.2-1.pgdg120+2) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit"


### Origin of the data
You can find more information on the dataset in the following links
- [Discogs](https://www.discogs.com/)
- [Discogs XML data dumps](http://data.discogs.com/)

If you do not want to use Docker or it does not work you can download the dataset from this [link](https://cloud.inf.ethz.ch/s/DtjCHTLRHT39BRN/download/discogs.dump.xz), see `postgres-init.sh` to see how to import it)

## Exercise 1: Explore the dataset
We want to first understand the dataset a bit better. You will find some queries below to help you explore the schema.

### List tables
The following query retrieves a list of tables in the database from a system table describing the current database.

In [4]:
%%sql 
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public';

 * postgresql://postgres:***@db:5432/postgres
22 rows affected.


table_name
companies
jobs
employees02
badges
comments
inventory
posthistory
postlinks
posts
tags


### List attributes/columns
The following query retrieves a list of columns from the tables in the database.

In [5]:
%%sql 
SELECT table_name, column_name, data_type, is_nullable, ordinal_position
FROM information_schema.columns
WHERE table_schema = 'public' AND table_name IN ('artists', 'released_by', 'releases', 'tracks')
AND table_name NOT LIKE 'pg_%'
ORDER BY table_name, ordinal_position;

 * postgresql://postgres:***@db:5432/postgres
17 rows affected.


table_name,column_name,data_type,is_nullable,ordinal_position
artists,artist_id,integer,NO,1
artists,name,character varying,YES,2
artists,realname,text,YES,3
artists,profile,text,YES,4
artists,url,text,YES,5
released_by,release_id,integer,NO,1
released_by,artist_id,integer,NO,2
releases,release_id,integer,NO,1
releases,released,date,NO,2
releases,title,text,NO,3


### Have a look at the datasets
The following simple query gives the first 5 rows of the `artists` dataset

In [6]:
%%sql
SELECT * FROM artists LIMIT 5;

 * postgresql://postgres:***@db:5432/postgres
5 rows affected.


artist_id,name,realname,profile,url
1,The Persuader,Jesper Dahlbäck,None,None
2,Mr. James Barth & A.D.,Cari Lekebusch & Alexi Delano,None,None
3,Josh Wink,Joshua Winkelman,"After forming [l=Ovum Recordings] as an independent label in October 1994 with former partner [a=King Britt], Josh recorded the cult classic 'Liquid Summer'. He went on to release singles for a wide variety of revered European labels ranging from Belgium's [l=R & S Records] to England's [l=XL Recordings]. In 1995, Wink became one of the first DJ-producers to translate his hard work into mainstream success when he unleashed a string of classics including 'Don't Laugh'¸ 'I'm Ready' and 'Higher State of Consciousness' that topped charts worldwide. More recently he has had massive club hits such as 'How's Your Evening So Far' and 'Superfreak' but he has also gained a lot of attention trough his remixes for [a=FC Kahuna], [a=Paul Oakenfold], [a=Ladytron], [a=Clint Mansell], [a=Sting] and [a=Depeche Mode], among others.",http://www.joshwink.com/
4,Johannes Heil,Johannes Heil,"Electronic music producer, musician and live performer, born 3 February 1978 near the town of Bad Nauheim, Germany. Founder of [l=JH] and [l=Metatron Recordings].",http://johannes-heil.com/
5,Heiko Laux,Heiko Laux,German DJ and producer based in Berlin. He is the founder of [l=Kanzleramt].,http://www.heiko-laux.com


Naturally we could write similar queries to better understand each of the other tables.

#### With what you now know about the datasets, try to answer the following questions

1. Which concepts are modelled in the dataset and how do they relate to each other? <b>Hint</b>: how do the tables connect logically?
2. Why do you think this shape (table) was chosen for the data and why not the other shapes?
3. In which normal forms are the corresponding relations?
4. How can we denormalise the data to make some queries more efficient? <b>Hint</b>: have a look at the queries in the next session of the exercises to see if adding some columns to some tables could reduce the need to `JOIN`.
5. What potential problems could result from adding redundancy?

### Solution

1. The dataset contains data from `discogs` and models musical artists, their physical releases, and the tracks on those releases. You can intuitively connect the tables with "artists release albums with tracks".

2. 
   - The number of concepts is limited, fixed, and well-defined.
   - The same is true for attributes of these concepts.
   - Attributes come from a well-defined domains with a fixed semantic (such as dates, e-mail addresses, ...).
   - Instances of concepts are in relation with each other, which may or may not be required to exist.
   - In short: we can define a schema, which the rest of the application (the website) relies on.


3. 
   - All relations are atomic so they are in 1NF.
   - `artists`, `releases` and `tracks` all have a singular ID primary key, so they at least 2NF. Upon inspection, there are no transitive dependencies, so they are in 3NF.
   - `released_by` does not contain non-key attributes, so it is in 3NF.


4. A very typical example of a query is finding all the releases by an artist's name. If we redundantly store the artist's ID and name in `releases`, the aforementioned query now only requires a `SELECT` instead of an expensive `JOIN`. As releases can be released by multiple artists, we could also extend this idea and break 1NF by storing a list of artist names instead. The tradeoff here is that every time we have to update an artist's name we have to find and update all releases by that artist. Considering that reads are very common and changes are rare, the option of denormalizing becomes relatively more attractive.

5. 
   - Insert anomaly: we could insert a release with a fake or incorrect artist name.
   - Update anomaly: if we update an artist's name but not all of the records in `releases`, we introduce an inconsistency into the dataset.

## Exercise 2: SQL warm-up
Now that we familiarised ourself with the tables and relationship, we will begin with several SQL queries to ease us back into the language.

<b>Practical tips:</b>
- You might want to begin by retrieving a few rows from each of the database tables to get a sense of what is stored. 
- When testing your queries, it is good practice to add a "LIMIT" clause to avoid inadvertedly retrieving hundreds of rows.

The following is an example query that contains some common SQL expressions. A complete list can be found at: https://www.postgresql.org/docs/current/sql-select.html

In [ ]:
%%sql
SELECT DISTINCT
    a.name AS column1,
    COUNT(t.track_id) AS column2,
    AVG(t.duration) AS column3
FROM
    artists a
    JOIN released_by rb USING(artist_id)
    JOIN releases r USING(release_id)
    JOIN tracks t USING(release_id)
WHERE
    t.duration > 123
    AND t.title != 'My Query'
    AND r.country = 'Switzerland'
GROUP BY
    a.artist_id, a.name
HAVING
    COUNT(t.track_id) > 0
ORDER BY
    column2 DESC,
    column3 DESC
LIMIT 5;

The following is a visual representation of the database schema for quick reference.

<img src="https://polybox.ethz.ch/index.php/s/8CqNffQrR0EDbuC/download" width=800/>

#### 1. Retrieve all releases that were released after January 1, 2017.

In [8]:
%%sql
SELECT title, released 
FROM releases
WHERE released > '2017-01-01';

 * postgresql://postgres:***@db:5432/postgres
18 rows affected.


title,released
"Kizomba Mix, Vol. 2 [2018] 2 CDs",2018-12-14
Sudd. Autumn Collection 03,2018-01-01
Technikal Support,2018-01-01
No-Harm,2018-01-01
Melburn / Into The Storm,2018-01-01
Invisible Agent 002,2018-01-01
Tribal Natty (Aphrodite Remix),2018-01-01
Shed01,2018-01-01
Jetlag Disco,2018-01-01
The Bad Behaviour E.P,2018-01-01


#### 2. Find all tracks with a duration longer than 7 hours. Assume the 'duration' column in the 'tracks' table is in seconds.

In [9]:
%%sql
SELECT title, duration
FROM tracks
WHERE duration > 25200;

 * postgresql://postgres:***@db:5432/postgres
3 rows affected.


title,duration
Rapper's Relight,25579
Dialectical Transformation III Peace In Rwanda Mix,27196
Live 1996.12.30.,31934


#### 3. Retrieve the titles of 5 releases along with the names of the artists who released them.

In [10]:
%%sql
SELECT r.title, a.name 
FROM releases r
JOIN released_by rb USING(release_id)
JOIN artists a USING(artist_id)
LIMIT 5;

 * postgresql://postgres:***@db:5432/postgres
5 rows affected.


title,name
Stockholm,The Persuader
Knockin' Boots Vol 2 Of 2,Mr. James Barth & A.D.
Profound Sounds Vol. 1,Josh Wink
Flowerhead,DATacide
Knockin' Boots (Vol 1 Of 2),Mr. James Barth & A.D.


#### 4. List each genre and the number of releases in that genre.

In [11]:
%%sql
SELECT genre, COUNT(release_id) AS number_of_releases
FROM releases
GROUP BY genre;

 * postgresql://postgres:***@db:5432/postgres
15 rows affected.


genre,number_of_releases
Blues,48
Brass & Military,4
Children's,6
Classical,257
Electronic,183766
"Folk, World, & Country",101
Funk / Soul,3674
Hip Hop,10598
Jazz,3325
Latin,153


#### 5. Identify the top 5 artists who have the most releases.

In [12]:
%%sql
SELECT a.name, COUNT(DISTINCT rb.release_id) AS release_count 
FROM artists a
JOIN released_by rb USING(artist_id)
GROUP BY a.artist_id, a.name
ORDER BY release_count DESC
LIMIT 5;

 * postgresql://postgres:***@db:5432/postgres
5 rows affected.


name,release_count
Various Artists,46123
Madonna,617
Pet Shop Boys,600
Faithless,336
Michael Jackson,332


#### 6. Find the artist who has the longest total duration of tracks across all their releases.

In [13]:
%%sql
SELECT a.name, SUM(t.duration) AS total_duration 
FROM artists a
JOIN released_by rb USING(artist_id)
JOIN tracks t USING(release_id)
GROUP BY a.artist_id, a.name
ORDER BY total_duration DESC
LIMIT 1;

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


name,total_duration
Various Artists,227180023


#### 7. Find how many releases that have tracks with duplicate titles.

In [14]:
%%sql
SELECT COUNT(DISTINCT release_id) FROM (
    SELECT release_id, title, COUNT(*) AS title_count
    FROM tracks
    GROUP BY release_id, title
    HAVING COUNT(*) > 1
) sub;

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


count
9046


#### 8. Retrieve the artists with the name of 'Coldplay'.

In [15]:
%%sql
SELECT *
FROM artists
WHERE name = 'Coldplay';

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


artist_id,name,realname,profile,url
29735,Coldplay,None,"Coldplay is an English rock band from London, England. They've been a band since January 16, 1998 when they lost a demotape competition on XFM in London. Philip Christopher Harvey is the band's manager. [b][u]Line-up:[/u][/b] Jonny Buckland (Jonathan Mark Buckland) - Guitar Will Champion (William Champion) - Drums Guy Berryman (Guy Rupert Berryman) - Bass Chris Martin (Christopher Anthony John Martin) - Vocals",http://coldplay.com/


#### 9. List the titles of all releases by that artist in alphabetical order.
<b>Hint</b>: Ignore the fact that different relases can have the same title.

In [16]:
%%sql
SELECT DISTINCT r.title
FROM artists a
JOIN released_by rb USING(artist_id)
JOIN releases r USING(release_id)
WHERE a.name = 'Coldplay'
ORDER BY r.title;

 * postgresql://postgres:***@db:5432/postgres
40 rows affected.


title
Acoustic
A Rush Of Blood To The Head
Boot Of Sound
Brothers & Sisters
Clocks
Clocks...
Clocks / Chime Trance Remixes
Clocks (Cosmos Rmx)
Clocks (Dean Coleman Remix)
Clocks (Planet Rockers Remixes)


#### 10. How many tracks from 'Coldplay' have position '1'?

In [17]:
%%sql
SELECT COUNT(t.track_id)
FROM artists a
JOIN released_by rb USING(artist_id)
JOIN releases r USING(release_id)
JOIN tracks t USING(release_id)
WHERE a.name = 'Coldplay' AND t.position = '1';

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


count
32


#### 11. List the titles of all releases by Coldplay that contain less than 2 tracks.

In [18]:
%%sql
SELECT DISTINCT r.title 
FROM artists a
JOIN released_by USING(artist_id)
JOIN releases r USING(release_id) 
JOIN tracks t USING(release_id)
WHERE a.name = 'Coldplay'
GROUP BY r.release_id, r.title
HAVING COUNT(*) < 2;

 * postgresql://postgres:***@db:5432/postgres
14 rows affected.


title
Boot Of Sound
Clocks
Clocks (Cosmos Rmx)
Clocks (Dean Coleman Remix)
Clocks (Remix)
God
In My Place
One I Love
Speed Of Sound (Karl G Remix)
Talk


#### 12. What is the average track duration?

In [19]:
%%sql
SELECT AVG(duration) FROM tracks;

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


avg
325.0749298696787647


#### 13. How many artists have released tracks longer than twice the average?

In [20]:
%%sql
SELECT COUNT(DISTINCT rb.artist_id)
FROM tracks t
JOIN releases r USING(release_id)
JOIN released_by rb USING(release_id)
where t.duration > 2*(SELECT AVG(duration) FROM tracks)

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


count
6386


## Exercise 3: more SQL
We will now see more complex SQL queries.

<b>Practical tips:</b>

When writing complex queries, you might want to split them into smaller parts by using <b>Common Table Expressions</b> (CTEs). A CTE is a named temporary result set that you can reference within statements (SELECT, INSERT, UPDATE, ... ). You can find more about CTEs at: https://www.postgresql.org/docs/current/queries-with.html

The following is an example of a query using two CTEs:

In [ ]:
%%sql
WITH countries AS (
    SELECT DISTINCT country FROM releases
),
genres AS (
    SELECT DISTINCT genre FROM releases
)
SELECT c.country as column1, g.genre as column2
FROM countries c, genres g
LIMIT 5;

In some exercises, you might also want to use <b>subqueries</b>. A subquery is a nested query, usually with the purpose of retrieving data that will be used in in the outer query. For instance, subqueries can appear in WHERE, FROM and SELECT clauses.

The following is an example of a query than includes a subquery:

In [ ]:
%%sql
SELECT release_id as column1 FROM (
    SELECT release_id, title, COUNT(*) FROM tracks
    GROUP BY release_id, title
    HAVING COUNT(*) > 1
) sub
LIMIT 5;

#### 1. What is the title of the album from 'Coldplay' with the most amount of tracks?

In [23]:
%%sql
WITH tracks_per_album AS(
    SELECT r.title AS album, COUNT(t.track_id) AS tracks
    FROM artists a
    JOIN released_by rb USING(artist_id)   
    JOIN releases r USING(release_id)
    JOIN tracks t USING(release_id)
    WHERE a.name = 'Coldplay'
    GROUP BY r.release_id, r.title
)

SELECT album, tracks
FROM tracks_per_album
WHERE tracks = (SELECT MAX(tracks)
                FROM tracks_per_album);

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


album,tracks
X&Y (Special Dutch Edition),19


alternatively,

In [24]:
%%sql
SELECT r.title AS album, COUNT(t.track_id) AS tracks
FROM artists a
JOIN released_by rb USING(artist_id)   
JOIN releases r USING(release_id)
JOIN tracks t USING(release_id)
WHERE a.name = 'Coldplay' 
GROUP BY r.release_id, r.title
ORDER BY COUNT(t.track_id) DESC
LIMIT 1;

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


album,tracks
X&Y (Special Dutch Edition),19


#### 2. What is the name of the first artist in alphabetical order with releases in the most genres. Please make sure to exclude "Various Artists".

In [25]:
%%sql
WITH artist_genre AS (
    SELECT COUNT(DISTINCT r.genre) AS c1, a.artist_id, a.name
    FROM artists a
    JOIN released_by rb USING(artist_id)
    JOIN releases r USING(release_id)
    GROUP BY a.artist_id
    ORDER BY c1 DESC, a.name ASC
),
artist_genre_cleaned AS (
    SELECT name
    FROM artist_genre
    WHERE name != 'Various Artists'
)
SELECT *
FROM artist_genre_cleaned
LIMIT 1;

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


name
Diana Ross


#### 3. In what year did they (the artist from the previous question) release their first album?

In [26]:
%%sql
WITH artist_genre AS (
    SELECT COUNT(DISTINCT r.genre) AS c1, a.artist_id, a.name
    FROM artists a
    JOIN released_by rb USING(artist_id)
    JOIN releases r USING(release_id)
    GROUP BY a.artist_id
    ORDER BY c1 DESC, a.name ASC
),
artist_genre_cleaned AS (
    SELECT name, artist_id
    FROM artist_genre
    WHERE name != 'Various Artists'
    LIMIT 1
)
SELECT EXTRACT(YEAR FROM released) AS year
FROM artist_genre_cleaned
JOIN released_by USING(artist_id)
JOIN releases USING(release_id)
ORDER BY year
LIMIT 1;

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


year
1967


alternatively,

In [27]:
%%sql
SELECT r.released
FROM releases r
JOIN released_by rb USING(release_id)
JOIN artists a USING(artist_id)
WHERE a.name = 'Diana Ross'
ORDER BY r.released ASC
LIMIT 1

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


released
1967-01-01


#### 4. How many artists have released an album with total track duration above twice the average total track duration?

<b>Hint</b>: this is not the same as exercise 2.13 since we are lookong at the <b>total</b> track duration of the album.

In [28]:
%%sql
WITH total_duration AS (
    SELECT release_id, SUM(duration) AS total_duration
    FROM tracks
    GROUP BY release_id
),
average_duration AS (
    SELECT AVG(total_duration) as avg_duration
    FROM total_duration
)

SELECT COUNT(DISTINCT(rb.artist_id))
FROM released_by rb
JOIN total_duration t USING(release_id)
WHERE t.total_duration > 2*(SELECT avg_duration FROM average_duration);

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


count
3388


#### 5. How many artists have both a release with a track longer than twice the average and one with total duration longer than twice the average?

<b>Hint</b>: you can use `INTERSECT` or `EXISTS` to write your query.

In [29]:
%%sql
WITH total_duration AS (
    SELECT release_id, SUM(duration) AS duration
    FROM tracks
    GROUP BY release_id
),
average AS (
    SELECT AVG(duration) AS avg_duration
    FROM total_duration
),
average_duration AS (
    SELECT AVG(CAST(duration AS FLOAT)) AS avg_track_duration
    FROM tracks
),
artist_intersection AS (
    SELECT artist_id
    FROM artists a
    JOIN released_by rb USING(artist_id)
    JOIN releases r USING(release_id)
    JOIN total_duration td USING(release_id)
    WHERE td.duration > 2*(SELECT avg_duration FROM average)
    INTERSECT
    SELECT artist_id
    FROM artists a
    JOIN released_by rb USING(artist_id)
    JOIN releases r USING(release_id)
    JOIN tracks t USING(release_id)
    WHERE t.duration > 2*(SELECT avg_track_duration FROM average_duration)
)

SELECT COUNT(*)
FROM artist_intersection;

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


count
1870


alternatively,

In [30]:
%%sql
WITH total_duration AS (
    SELECT release_id, SUM(duration) as duration
    FROM tracks
    GROUP BY release_id
),
total_average_duration AS (
    SELECT AVG(duration) as avg_duration
    FROM total_duration
),
-- IDs of artists that have 1 track > twice the total average
artists_total_avg AS (
    SELECT DISTINCT b.artist_id
    FROM released_by b
    INNER JOIN total_duration t USING(release_id)
    WHERE duration > 2 * (SELECT avg_duration FROM total_average_duration)
),
-- IDs of artists that have 1 track > the normal average
artists_avg AS (
    SELECT DISTINCT rb.artist_id FROM tracks t
    INNER JOIN releases r USING(release_id)
    INNER JOIN released_by rb USING(release_id)
    WHERE t.duration > 2 * (SELECT AVG(duration) FROM tracks)
)
-- Count the intersection (remember that intersect applies to everything in the select)
SELECT COUNT(*)
FROM (
    SELECT artist_id
    FROM artists_total_avg
    INTERSECT
    SELECT artist_id
    FROM artists_avg
)

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


count
1870


#### 6. Show the artists have more than 200 releases in total but have no releases with the genre 'Pop' in reversed alphabetical order.

In [31]:
%%sql
WITH big_guys AS (
    SELECT artist_id, name, COUNT(release_id) AS c
    FROM artists
    NATURAL JOIN released_by
    NATURAL JOIN releases
    GROUP BY artist_id, name
    HAVING COUNT(release_id) > 200
)

SELECT name
FROM big_guys
WHERE artist_id NOT IN (
    SELECT artist_id
    FROM artists b
    NATURAL JOIN released_by
    NATURAL JOIN releases
    WHERE genre = 'Pop'
)
ORDER BY name DESC;

 * postgresql://postgres:***@db:5432/postgres
11 rows affected.


name
Underworld
The Shamen
The Art Of Noise
Technotronic
Tangerine Dream
Pet Shop Boys
Orbital
Kool & The Gang
Faithless
Beastie Boys


alternatively,

In [32]:
%%sql
WITH pop_list AS (
    SELECT DISTINCT rb.artist_id
    FROM released_by rb
    INNER JOIN releases r USING(release_id)
    WHERE r.genre = 'Pop'
)

SELECT a.name
FROM released_by rb
INNER JOIN artists a USING(artist_id)
WHERE rb.artist_id NOT IN (SELECT * FROM pop_list)
GROUP BY rb.artist_id, a.name
HAVING COUNT(rb.release_id) > 200
ORDER BY a.name DESC;

 * postgresql://postgres:***@db:5432/postgres
11 rows affected.


name
Underworld
The Shamen
The Art Of Noise
Technotronic
Tangerine Dream
Pet Shop Boys
Orbital
Kool & The Gang
Faithless
Beastie Boys


## Exercise 4: Discuss query patterns and language features of SQL
1. What patterns did you use in many of the queries above? 

2. What is the usual pattern of an SQL query? Which operations happen pre-grouping and which ones post-grouping?

3. What makes SQL a declarative language and what advantages does that have?

4. What makes SQL a functional language and what advantages does that have?

5. How would the denormalization we talked about previously simplify the queries?

### Solution

1. Most queries consist of the following basic operations. They will reoccur throughout the whole semester. Watch out for them!
  * **Select**: select a subset of the rows/data records/items.
  * **Project**: select a subset of the properties/ attributes/columns.
  * **Join**: bring two datasets together based on a common attribute.
  * **Group**: divide the items/ rows/records into groups and summarize each group with a single value.
  * **Order**: order the items according to some criteria.

2. The usual syntax of a SQL query is `SELECT, FROM, WHERE, GROUP BY, HAVING, ORDER BY, LIMIT, OFFSET`. Ignoring the last 3 that only influence how the result is presented, `FROM` and `WHERE` happen before the grouping and `HAVING` and `SELECT` after.
  
3. We only describe *what* we want, not how this should be computed. We *declare* what our intent is. This shifts the implementation effort from the programmer to the database system. The hope is that the system has more information at hand, such as data size, data distribution, information about the hardware, in order to choose the best way to compute the result. This results into efficient computation with little effort from the programmer.
  
4. SQL is functional because results of a query can be used as input of another query, either in form of tables or in form of scalars. This makes SQL expressive.

5. If we denormalised the data and unify some of the tables, we no longer need to join them every time in order to get our results.

## Exercise 5: Limits of SQL (optional)
Explain what the following query does.
<b>Hints</b>: The query treats the data as if it was in graph shape.

In [33]:
%%sql
WITH RECURSIVE
    X AS (SELECT 3 AS Value),
    artist_releases AS (
        SELECT artists.artist_id, artists.name, releases.release_id, releases.title
        FROM artists, released_by, releases
        WHERE artists.artist_id = released_by.artist_id
        AND released_by.release_id = releases.release_id
    ),
    collaborations AS (
        SELECT DISTINCT ar1.artist_id AS left_id, ar1.name AS left_name, 
                ar2.artist_id AS right_id, ar2.name AS right_name, 1 AS distance
        FROM artist_releases AS ar1, artist_releases AS ar2
        WHERE ar1.release_id = ar2.release_id
        AND ar1.artist_id != ar2.artist_id
    ),
    X_hop_collaborations AS (
        SELECT * FROM collaborations  -- base case
        UNION
        SELECT c1.left_id, c1.left_name, c2.right_id, c2.right_name, c1.distance + 1 AS distance
        FROM X_hop_collaborations AS c1
        JOIN collaborations c2 ON c1.right_id = c2.left_id
        WHERE c1.distance < (SELECT * FROM X) AND c1.left_id != c2.right_id
    )
SELECT * 
FROM X_hop_collaborations
WHERE left_name = 'Coldplay'
ORDER BY distance, right_name;

 * postgresql://postgres:***@db:5432/postgres
23 rows affected.


left_id,left_name,right_id,right_name,distance
29735,Coldplay,1654,DK,1
29735,Coldplay,392179,G Synth,1
29735,Coldplay,10916,Jan Johnston,1
29735,Coldplay,1279,Orbital,1
29735,Coldplay,10785,Angelo Badalamenti,2
29735,Coldplay,11101,Cosmic Gate,2
29735,Coldplay,7090,Freefall,2
29735,Coldplay,2604010,Jada (7),2
29735,Coldplay,18836,Kirk Hammett,2
29735,Coldplay,11749,Knuckleheadz,2


### Solution
The query interprets the database as an undirected graph where *Artists* are nodes and edges exist between artists that have released an album together (i.e., collaborated). It then returns all of the artists which are at most `X` hops from a given artist in the graph. Even if the problem is simple when phrased with graph terminology, we can see that the query itself is quite complex: evidently, **SQL is not built for graph analysis!**. In the later weeks of the course, we will see tools that make graph analysis much more intuitive.

To understand why the query does what it does, think about how we can find all artists exactly `X` hops from a given artist. Firstly, we need to be able to easily retrieve all *directly* collaborating artists, i.e., the set of edges:

In [34]:
%%sql
WITH artist_releases AS (
    SELECT artists.artist_id, artists.name, releases.release_id, releases.title
    FROM artists 
    JOIN released_by USING(artist_id)
    JOIN releases USING (release_id)
)
SELECT DISTINCT ar1.artist_id AS left_id, ar1.name AS left_name, 
        ar2.artist_id AS right_id, ar2.name AS right_name, 1 AS distance
FROM artist_releases AS ar1, artist_releases AS ar2
WHERE ar1.release_id = ar2.release_id
AND ar1.artist_id != ar2.artist_id
LIMIT 10;

 * postgresql://postgres:***@db:5432/postgres
10 rows affected.


left_id,left_name,right_id,right_name,distance
7298,The Stone Roses,4097,Whodini,1
50051,Seven (2),290,Hidden Agenda,1
106085,Arpeggio (2),138075,Quartario,1
240418,Cantero,182615,Bongoloverz,1
28732,Scrappy,928337,Wet Boxx,1
10677,Soulman,4967,Lighter Thief,1
14014,Common,48284,The Last Poets,1
169612,Holmek,169611,Demon Queen,1
128046,Woody (5),179180,DJ Exodus,1
7300,Jesus Loves You,21032,Boy George,1


where we first join artists with their releases under `artist_releases`, and then join `artist_releases` with itself to find all pairs of artists which have released the same `release_id`. We limit the query results to 10 for demonstration purposes.

We give this query the alias `collaborations` and join it with itself on a mutual `artist_id` to find all collaborations with two hops:

In [35]:
%%sql
WITH artist_releases AS (
    SELECT artists.artist_id, artists.name, releases.release_id, releases.title
    FROM artists 
    JOIN released_by USING(artist_id)
    JOIN releases USING (release_id)
),
collaborations AS (
    SELECT DISTINCT ar1.artist_id AS left_id, ar1.name AS left_name, 
            ar2.artist_id AS right_id, ar2.name AS right_name, 1 AS distance
    FROM artist_releases AS ar1, artist_releases AS ar2
    WHERE ar1.release_id = ar2.release_id
    AND ar1.artist_id != ar2.artist_id
),
two_hop_collaborations AS (
    SELECT DISTINCT c1.left_id, c1.left_name, c2.right_id, c2.right_name
    FROM collaborations c1, collaborations c2
    WHERE c1.right_id = c2.left_id
)
SELECT *
FROM two_hop_collaborations
WHERE left_name = 'Coldplay'
ORDER BY right_name
LIMIT 10;

 * postgresql://postgres:***@db:5432/postgres
8 rows affected.


left_id,left_name,right_id,right_name
29735,Coldplay,10785,Angelo Badalamenti
29735,Coldplay,29735,Coldplay
29735,Coldplay,11101,Cosmic Gate
29735,Coldplay,7090,Freefall
29735,Coldplay,2604010,Jada (7)
29735,Coldplay,18836,Kirk Hammett
29735,Coldplay,11749,Knuckleheadz
29735,Coldplay,166466,Skip Raiders


We can iteratively add indirections with another join like so:

In [36]:
%%sql
WITH artist_releases AS (
    SELECT artists.artist_id, artists.name, releases.release_id, releases.title
    FROM artists 
    JOIN released_by USING(artist_id)
    JOIN releases USING (release_id)
),
collaborations AS (
    SELECT DISTINCT ar1.artist_id AS left_id, ar1.name AS left_name, 
            ar2.artist_id AS right_id, ar2.name AS right_name, 1 AS distance
    FROM artist_releases AS ar1, artist_releases AS ar2
    WHERE ar1.release_id = ar2.release_id
    AND ar1.artist_id != ar2.artist_id
),
two_hop_collaborations AS (
    SELECT DISTINCT c1.left_id, c1.left_name, c2.right_id, c2.right_name
    FROM collaborations c1, collaborations c2
    WHERE c1.right_id = c2.left_id
),
three_hop_collaborations AS (
    SELECT DISTINCT c1.left_id, c1.left_name, c2.right_id, c2.right_name
    FROM two_hop_collaborations c1, collaborations c2
    WHERE c1.right_id = c2.left_id
),
four_hop_collaborations AS (
    SELECT DISTINCT c1.left_id, c1.left_name, c2.right_id, c2.right_name
    FROM three_hop_collaborations c1, collaborations c2
    WHERE c1.right_id = c2.left_id
)
SELECT *
FROM four_hop_collaborations
WHERE left_name = 'Coldplay'
ORDER BY right_name
LIMIT 10;

 * postgresql://postgres:***@db:5432/postgres
10 rows affected.


left_id,left_name,right_id,right_name
29735,Coldplay,10785,Angelo Badalamenti
29735,Coldplay,7623,Charly Lownoise
29735,Coldplay,29735,Coldplay
29735,Coldplay,11101,Cosmic Gate
29735,Coldplay,129695,DJ Boom (2)
29735,Coldplay,12683,DJ Hooligan
29735,Coldplay,17240,DJ JamX
29735,Coldplay,170,DJ Sneak
29735,Coldplay,29121,DJ Work!
29735,Coldplay,8353,DuMonde


he problem with this approach is that the query *structure* depends on the number of hops you want to permit.

The query from the question, however, is independent of this number. You can just change `X` and leave the rest as it is. The trick is that it is a *recursive* SQL statement. Roughly speaking, a recursive statement consists of a base case statement connected via `UNION` (a concatenation of relations) with a statement the refers to the original statement. The recursive statement is executed recursively until it produces an empty result.

The query uses this mechanism to write `X_hop_collaborations` (repeated in isolation below): its base case are the direct `collaborations`. Each iteration joins the `collaborations` relation to find artists within one more hop for each artist already found. It keeps track of the distance from the original post by incrementing the value of `distance` by one in each iteration. When that attribute reaches the value of `X`, no records are produced by the recursive case, so recursion ends.

Consider: we have to join `collaborations` with an exponentially growing relation for each hop. In addition to the query being difficult to write, it becomes very inefficient as `X` increases.